### Part 1
Read the NeTEx files and extract all unique journeys on a line. This is every journeynumber per operatingday. The TimeDemandTypeRef is found for every journey. This is later on used to connect each journey with the scheduled driving times between the stops.

In [ ]:
import re
import xml.etree.ElementTree as ET
import pandas as pd
import datetime as dt
from pathlib import Path

pd.set_option("display.max_colwidth", None)

files = [
    '../../../data/NeTEx/NeTEx_CXX_SRE_20250527_2025-06-01_202500014_baseline.xml',
    '../../../data/NeTEx/NeTEx_CXX_SRE_20250702_2025-07-06_202500015_baseline.xml',
    '../../../data/NeTEx/NeTEx_CXX_SRE_20250717_2025-07-20_202500016_baseline.xml',
    '../../../data/NeTEx/NeTEx_CXX_SRE_20250731_2025-08-03_202500018_baseline.xml',
    '../../../data/NeTEx/NeTEx_CXX_SRE_20250812_2025-08-17_202500019_baseline.xml',
]

LINE_PREFIX = "CXX:ServiceJourney:L401-"

def parse_filename_meta(path: str):
    """
    From filename like:
    NeTEx_CXX_SRE_20250527_2025-06-01_202500014_baseline.xml
    -> export_created_yyyymmdd=20250527, export_validfrom=2025-06-01, export_version=202500014
    """
    name = Path(path).name
    m = re.search(r'NeTEx_.*_(\d{8})_(\d{4}-\d{2}-\d{2})_(\d{9})_baseline\.xml', name)
    if not m:
        return None, None, None
    created = dt.datetime.strptime(m.group(1), "%Y%m%d").date()
    validfrom = dt.date.fromisoformat(m.group(2))
    version = int(m.group(3))
    return created, validfrom, version

def parse_dep_time(dep_time_str: str):
    try:
        return dt.time.fromisoformat(dep_time_str)
    except Exception:
        return None

def build_tables(root):
    # namespace
    ns = ''
    if root.tag.startswith('{'):
        ns = root.tag.split('}')[0].strip('{')
        nsmap = {'ns': ns}
    else:
        nsmap = {}

    def t(tag: str) -> str:
        return f'{{{ns}}}{tag}' if ns else tag

    # AvailabilityCondition lookup
    availability = {}
    for ac in root.findall(f'.//{t("AvailabilityCondition")}', nsmap):
        ac_id = ac.get('id')
        if not ac_id:
            continue
        from_dt = (ac.findtext(t('FromDate')) if ns else ac.findtext('FromDate')) or ""
        to_dt   = (ac.findtext(t('ToDate')) if ns else ac.findtext('ToDate')) or ""
        bits_raw = (ac.findtext(t('ValidDayBits')) if ns else ac.findtext('ValidDayBits')) or ""
        bits = "".join(bits_raw.split())
        availability[ac_id] = {"from_dt": from_dt, "to_dt": to_dt, "bits": bits}

    def expand_availability_dates(ac_id: str) -> list[dt.date]:
        info = availability.get(ac_id)
        if not info:
            return []
        from_dt, to_dt, bits = info["from_dt"], info["to_dt"], info["bits"]
        if not from_dt or not to_dt or not bits:
            return []

        start = dt.date.fromisoformat(from_dt[:10])
        to_date = dt.date.fromisoformat(to_dt[:10])

        # handle midnight end boundary as end-exclusive (common)
        to_time_part = to_dt[11:19] if len(to_dt) >= 19 else None
        end_inclusive = to_date - dt.timedelta(days=1) if to_time_part == "00:00:00" else to_date
        if end_inclusive < start:
            return []

        days_inclusive = (end_inclusive - start).days + 1

        # Case A: 7-bit weekday mask Mon..Sun
        if len(bits) == 7:
            out = []
            cur = start
            while cur <= end_inclusive:
                if bits[cur.weekday()] == "1":
                    out.append(cur)
                cur += dt.timedelta(days=1)
            return out

        # Case B: per-day bitstring from start
        n = min(days_inclusive, len(bits))
        return [start + dt.timedelta(days=i) for i in range(n) if bits[i] == "1"]

    # Extract ServiceJourneys for the line
    rows = []
    for sj in root.findall(f'.//{t("ServiceJourney")}', nsmap):
        sj_id = sj.get('id', '')
        if not sj_id.startswith(LINE_PREFIX):
            continue

        dep_time = sj.findtext(t('DepartureTime')) if ns else sj.findtext('DepartureTime')
        dep_t = parse_dep_time(dep_time or "")
        if dep_t is None:
            continue

        # JourneyNumber
        private_code = sj.find(t('PrivateCode')) if ns else sj.find('PrivateCode')
        journey_number = (
            private_code.text
            if private_code is not None and private_code.get('type') == 'JourneyNumber'
            else None
        )
        if not journey_number:
            continue

        # TimeDemandTypeRef
        tdt = sj.find(t('TimeDemandTypeRef')) if ns else sj.find('TimeDemandTypeRef')
        tdt_ref = tdt.get('ref') if tdt is not None else None
        if not tdt_ref:
            continue

        # ServiceJourneyPatternRef (helps dedupe correctly)
        sjp = sj.find(t('ServiceJourneyPatternRef')) if ns else sj.find('ServiceJourneyPatternRef')
        sjp_ref = sjp.get('ref') if sjp is not None else None

        # AvailabilityConditionRef
        ac = sj.find(f'.//{t("AvailabilityConditionRef")}') if ns else sj.find('.//AvailabilityConditionRef')
        ac_ref = ac.get('ref') if ac is not None else None
        if not ac_ref:
            continue

        dates = expand_availability_dates(ac_ref)

        for d in dates:
            rows.append({
                "ServiceJourneyId": sj_id,
                "ServiceJourneyPatternRef": sjp_ref,
                "JourneyNumber": journey_number,
                "TimeDemandTypeRef": tdt_ref,
                "AvailabilityConditionRef": ac_ref,
                "DepartureDateTime": dt.datetime.combine(d, dep_t),
            })

    return pd.DataFrame(rows)

all_expanded = []

for path in files:
    created, validfrom, file_version = parse_filename_meta(path)

    tree = ET.parse(path)
    root = tree.getroot()
    df_expanded = build_tables(root)

    df_expanded["SourceFile"] = Path(path).name
    df_expanded["ExportCreatedDate"] = created
    df_expanded["ExportValidFrom"] = validfrom
    df_expanded["ExportVersion"] = file_version

    all_expanded.append(df_expanded)

combined = pd.concat(all_expanded, ignore_index=True)

# --- Keep only the latest export for overlapping trips ---
# Define “same trip” key:
# pattern + journey number + dated departure is usually stable enough.
dedupe_key = ["ServiceJourneyPatternRef", "JourneyNumber", "DepartureDateTime"]

combined_latest = (
    combined.sort_values(
        ["DepartureDateTime", "ExportCreatedDate", "ExportVersion"],
        ascending=[True, False, False]
    )
    .drop_duplicates(subset=dedupe_key, keep="first")
    .reset_index(drop=True)
)

# print("Combined rows:", len(combined))
# print("After keeping latest per trip:", len(combined_latest))

combined_latest.to_csv('dova_l401_servicejourneys_expanded_latest.csv', index=False)
combined_latest.head(20)



### Part 2
The goal of this part is to get the scheduled driving times between all stops for all TimeDemandTypeRefs. This can then later be connected to the journeys.

In [ ]:
# Extract TimeDemandType -> ordered stop-to-stop runtimes, collapsing TimingPoint segments:
#   Stop -> TimingPoint -> Stop  ==> Stop -> Stop with summed runtime
# Also fixes scrambled XML order by ordering edges into a path using TimingLinkRef ids.

import xml.etree.ElementTree as ET
import re
import pandas as pd

def parse_duration_to_seconds(iso: str) -> int | None:
    if not iso:
        return None
    iso = iso.strip()
    m = re.fullmatch(r'PT(?:(\d+)H)?(?:(\d+)M)?(?:(\d+)S)?', iso)
    if not m:
        return None
    h = int(m.group(1) or 0)
    mi = int(m.group(2) or 0)
    s = int(m.group(3) or 0)
    return h * 3600 + mi * 60 + s

def local(tag: str) -> str:
    return tag.split("}", 1)[1] if "}" in tag else tag

def order_raw_edges_into_path(raw_edges: list[tuple[str, str, int, str]]):
    """
    raw_edges: list of (from_ref, to_ref, runtime_sec, link_id)
    Returns ordered list of edges in path order (same tuple form).
    Assumes no branches.
    """
    if not raw_edges:
        return []

    nodes = set()
    in_deg = {}
    out_map = {}

    for a, b, rt, lid in raw_edges:
        nodes.add(a); nodes.add(b)
        in_deg[b] = in_deg.get(b, 0) + 1
        in_deg.setdefault(a, in_deg.get(a, 0))
        out_map.setdefault(a, []).append((b, rt, lid))

    starts = [n for n in nodes if in_deg.get(n, 0) == 0]
    start = starts[0] if starts else raw_edges[0][0]

    ordered = []
    cur = start
    used_links = set()
    max_steps = len(raw_edges) + 5  # safety

    steps = 0
    while steps < max_steps:
        steps += 1
        nxt = None
        for b, rt, lid in out_map.get(cur, []):
            if lid not in used_links:
                nxt = (b, rt, lid)
                break
        if nxt is None:
            break
        b, rt, lid = nxt
        used_links.add(lid)
        ordered.append((cur, b, rt, lid))
        cur = b
        if len(ordered) == len(raw_edges):
            break

    return ordered

def collapse_timingpoints_to_stops(ordered_edges: list[tuple[str, str, int, str]]):
    """
    ordered_edges: (from_ref, to_ref, runtime_sec, link_id) in path order.
    refs may be ScheduledStopPoint or TimingPoint.

    Collapses:
      Stop -> TP -> Stop into Stop -> Stop (sum runtimes)
    Also handles endpoint TP segments by attaching them to adjacent stop segment when possible.

    Returns:
      stop_edges: list of (from_stop_ref, to_stop_ref, runtime_sec, link_ids_used)
    """
    def is_stop(x: str) -> bool:
        return x is not None and (":ScheduledStopPoint:" in x)

    def is_tp(x: str) -> bool:
        return x is not None and (":TimingPoint:" in x)

    stop_edges = []
    i = 0
    while i < len(ordered_edges):
        a, b, rt, lid = ordered_edges[i]

        # Stop -> TimingPoint
        if is_stop(a) and is_tp(b):
            # If next edge is TimingPoint -> Stop, collapse both into Stop->Stop
            if i + 1 < len(ordered_edges):
                a2, b2, rt2, lid2 = ordered_edges[i + 1]
                if a2 == b and is_stop(b2):
                    stop_edges.append((a, b2, rt + rt2, [lid, lid2]))
                    i += 2
                    continue
            # Endpoint Stop->TP without a TP->Stop after it: skip (or keep separately if you want)
            i += 1
            continue

        # TimingPoint -> Stop (start-of-trip timing point)
        if is_tp(a) and is_stop(b):
            # If next edge starts at that stop, attach this runtime to the next stop->(something) edge
            if i + 1 < len(ordered_edges):
                a2, b2, rt2, lid2 = ordered_edges[i + 1]
                # Ideally Stop -> Stop next
                if a2 == b and is_stop(a2) and is_stop(b2):
                    stop_edges.append((a2, b2, rt + rt2, [lid, lid2]))
                    i += 2
                    continue
                # Or Stop -> TP next, which will then be handled by Stop->TP->Stop collapse
                if a2 == b and is_stop(a2) and is_tp(b2):
                    # Let the Stop->TP->Stop collapse handle it, but carry runtime forward by
                    # replacing next edge runtime with rt+rt2:
                    ordered_edges[i + 1] = (a2, b2, rt + rt2, lid2)
                    i += 1
                    continue
            i += 1
            continue

        # Normal Stop -> Stop
        if is_stop(a) and is_stop(b):
            stop_edges.append((a, b, rt, [lid]))
            i += 1
            continue

        # TP -> TP or unknown: skip
        i += 1

    return stop_edges

def extract_tdt_runtimes_and_stops_collapsed(xml_path: str) -> dict[str, dict]:
    tree = ET.parse(xml_path)
    root = tree.getroot()

    # namespace detect
    ns = ''
    if root.tag.startswith('{'):
        ns = root.tag.split('}')[0].strip('{')
        nsmap = {'ns': ns}
    else:
        nsmap = {}

    def t(tag: str) -> str:
        return f'{{{ns}}}{tag}' if ns else tag

    # Index elements by @id so TimingLinkRef can be resolved quickly
    id_index = {}
    for el in root.iter():
        eid = el.get("id")
        if eid:
            id_index[eid] = el

    def get_from_to_refs(link_el):
        if link_el is None:
            return None, None
        from_ref = None
        to_ref = None
        for child in link_el.iter():
            cl = local(child.tag)
            if cl in ("FromPointRef", "FromStopPointInJourneyPatternRef") and child.get("ref"):
                from_ref = child.get("ref")
            elif cl in ("ToPointRef", "ToStopPointInJourneyPatternRef") and child.get("ref"):
                to_ref = child.get("ref")
            if from_ref and to_ref:
                break
        return from_ref, to_ref

    out = {}

    for tdt_el in root.findall(f'.//{t("TimeDemandType")}', nsmap):
        tdt_id = tdt_el.get("id")
        if not tdt_id:
            continue

        raw_edges = []  # (from_ref, to_ref, runtime_sec, link_id)

        for jrt in tdt_el.findall(f'.//{t("JourneyRunTime")}', nsmap):
            sec = parse_duration_to_seconds(jrt.findtext(t("RunTime")))
            if sec is None:
                continue

            tlr = jrt.find(t("TimingLinkRef"))
            link_ref = tlr.get("ref") if tlr is not None else None
            if not link_ref:
                continue

            link_el = id_index.get(link_ref)
            fref, toref = get_from_to_refs(link_el)
            if not fref or not toref:
                continue

            raw_edges.append((fref, toref, sec, link_ref))

        # 1) order the raw timing-link edges into path order
        ordered_edges = order_raw_edges_into_path(raw_edges)

        # 2) collapse TimingPoint segments into stop-to-stop segments
        stop_edges = collapse_timingpoints_to_stops(ordered_edges)

        # Build StopCodes + RunTimesSeconds arrays
        if stop_edges:
            stop_codes = [stop_edges[0][0]] + [e[1] for e in stop_edges]
            run_times = [e[2] for e in stop_edges]
        else:
            stop_codes = []
            run_times = []

        out[tdt_id] = {
            "RunTimesSeconds": run_times,
            "StopCodes": stop_codes,
            "n_raw_segments": len(raw_edges),
            "n_ordered_segments": len(ordered_edges),
            "n_collapsed_segments": len(run_times),
        }

    return out

# ---------------------------------------------------------------------
# BUILD MASTER MAP ACROSS MULTIPLE FILES (don't overwrite non-empty with empty)
# Requires:
#   files: list[str]          # your 5 NeTEx XML paths
#   combined_latest: DataFrame with TimeDemandTypeRef (from your earlier step)
# ---------------------------------------------------------------------

tdt_master = {}
for path in files:
    tdt_map = extract_tdt_runtimes_and_stops_collapsed(path)
    for k, v in tdt_map.items():
        new_len = len(v.get("RunTimesSeconds") or [])
        old_len = len(tdt_master.get(k, {}).get("RunTimesSeconds") or [])
        if k not in tdt_master or (old_len == 0 and new_len > 0):
            tdt_master[k] = v

# Build df for the TimeDemandTypes actually used
unique_tdts = combined_latest["TimeDemandTypeRef"].dropna().unique()

rows = []
for ref in unique_tdts:
    entry = tdt_master.get(ref)
    rows.append({
        "TimeDemandTypeRef": ref,
        "RunTimesSeconds": entry["RunTimesSeconds"] if entry else None,
        "StopCodes": entry["StopCodes"] if entry else None,
        "n_segments": len(entry["RunTimesSeconds"]) if entry else 0,
        "n_stops": len(entry["StopCodes"]) if entry else 0,
        "n_raw_segments": entry.get("n_raw_segments") if entry else None,
        "n_ordered_segments": entry.get("n_ordered_segments") if entry else None,
        "missing_definition": entry is None
    })

tdt_df = (
    pd.DataFrame(rows)
      .sort_values(["missing_definition", "TimeDemandTypeRef"])
      .reset_index(drop=True)
)
# keep only last numeric part of StopCodes (e.g., "CXX:ScheduledStopPoint:68601010" -> "68601010") ---
def _last_number(x):
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return None
    s = str(x)
    m = re.search(r'(\d+)\s*$', s)  # trailing digits
    return m.group(1) if m else s   # fallback: leave as-is if no trailing digits

def _clean_stopcodes_list(lst):
    if lst is None or (isinstance(lst, float) and pd.isna(lst)):
        return None
    if not isinstance(lst, list):
        return lst
    return [_last_number(v) for v in lst]

tdt_df["StopCodes"] = tdt_df["StopCodes"].apply(_clean_stopcodes_list)

# Sanity: for non-empty, stops should be segments + 1
bad = tdt_df.query("n_segments > 0 and n_stops != n_segments + 1")
print("Broken segment/stop alignment:", len(bad))

tdt_df[
    ["TimeDemandTypeRef", "RunTimesSeconds", "StopCodes"]
].to_csv('dova_l401_timedemandtype_runtimes_collapsed.csv', index=False)
tdt_df.head(10)

### Part 3
In this part the data from the previous parts are connected. The final csv file contains the following columns: JourneyNumber,DepartureDateTime,RunTimesSeconds,StopCodes. This data can be connected with the KV6 messages. This connection happens based on the JourneyNumber and departure date. It can explain for every message in the KV6 dataset what the planned driving times are.

In [ ]:
# Create final dataset
import pandas as pd
from pandas import read_csv

df1 = read_csv('dova_l401_servicejourneys_expanded_latest.csv')
df2 = read_csv('dova_l401_timedemandtype_runtimes_collapsed.csv')

# Final format: merge df1 and df2 on TimeDemandTypeRef. Include per row: JourneyNumber, DepartureDateTime, OperatingDay, RunTimesSeconds, StopCodes
final_df = pd.merge(
    df1,
    df2,
    on='TimeDemandTypeRef',
    how='left'
 )

final_df["DepartureDateTime"] = pd.to_datetime(final_df["DepartureDateTime"], errors="coerce")
final_df["OperatingDay"] = final_df["DepartureDateTime"].dt.strftime("%Y-%m-%d")

final_df = final_df[["TimeDemandTypeRef", "JourneyNumber", "DepartureDateTime", "OperatingDay", "RunTimesSeconds", "StopCodes"]]

final_df.to_csv('dova_l401_data.csv', index=False)
final_df.head(20)

### Part 4
Merge Line 401 messages exp with the dova dataset from step 3. Joining takes place based on JourneyNumber and OperatingDay. The final df contains all data from messages_exp with added the RunTimeSeconds.

In [ ]:
# Part 4 - Merge Line 401 messages_exp with DOVA dataset (Part 3)
from pathlib import Path
import pandas as pd

dova_path = Path('dova_l401_data.csv')
msg_path = Path('../../input/line401_combinations_exp.csv')

df_dova = pd.read_csv(dova_path)
df_msg = pd.read_csv(msg_path)

def _col_lookup(df: pd.DataFrame) -> dict[str, str]:
    return {str(c).strip().lower(): c for c in df.columns}

# --- Messages: standardize JourneyNumber + OperatingDay ---
msg_cols = _col_lookup(df_msg)
jn_col = msg_cols.get('journeynumber')
if not jn_col:
    raise KeyError("messages_exp is missing JourneyNumber (or JourneyNumberr)")
df_msg['JourneyNumber'] = df_msg[jn_col].astype(str)

op_col = msg_cols.get('operatingday')
if op_col:
    df_msg['OperatingDay'] = pd.to_datetime(df_msg[op_col], errors='coerce').dt.strftime('%Y-%m-%d')
else:
    dt_candidates = ['DepartureDateTime', 'departuredatetime', 'Timestamp', 'timestamp', 'MessageDateTime', 'messagedatetime', 'DateTime', 'datetime', 'TimeStamp', 'timestamp']
    dt_col = next((msg_cols.get(c.lower()) for c in dt_candidates if msg_cols.get(c.lower())), None)
    if not dt_col:
        raise KeyError("messages_exp is missing OperatingDay and no known datetime column was found")
    df_msg['OperatingDay'] = pd.to_datetime(df_msg[dt_col], errors='coerce').dt.strftime('%Y-%m-%d')

# --- DOVA: standardize JourneyNumber + OperatingDay ---
dova_cols = _col_lookup(df_dova)
if 'journeynumber' not in dova_cols or 'operatingday' not in dova_cols:
    raise KeyError("dova_l401_data.csv must contain JourneyNumber and OperatingDay")
df_dova['JourneyNumber'] = df_dova[dova_cols['journeynumber']].astype(str)
df_dova['OperatingDay'] = pd.to_datetime(df_dova[dova_cols['operatingday']], errors='coerce').dt.strftime('%Y-%m-%d')

# Merge (keep all messages rows; attach planned runtimes)
merged_df = df_msg.merge(
    df_dova[['JourneyNumber', 'OperatingDay','StopCodes', 'RunTimesSeconds', 'TimeDemandTypeRef']],
    on=['JourneyNumber', 'OperatingDay'],
    how='left'
)

merged_df.drop(["JourneyNumber", "OperatingDay"], axis=1, inplace=True)
merged_df.to_csv('line401_combinations_exp_with_dova.csv', index=False)
merged_df.tail(30)

In [ ]:
# Extract the running time segments for a specific journey:
import pandas as pd

df = pd.read_csv('line401_combinations_exp_with_dova.csv');
# Filter dataset for 02-06-2025
df = df[(df['operatingday'] == '2025-06-09') & (df['to_station'] == 1) & (df['journeynumber'] == 7075)]
df.head(20);

# These running times can be used for example for predictions of delays.